In [1]:
import torch
import torch.nn as nn
import numpy as np
import json
"""
# Define the model class based on the provided LIMU-BERT structure
class LIMUBERTModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LIMUBERTModel, self).__init__()
        self.embed = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.Transformer(hidden_dim, nhead=8, num_encoder_layers=6)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

        """

class LIMUBERTModel(nn.Module):
    def __init__(self):
        super(LIMUBERTModel, self).__init__()
        self.embed = nn.Linear(6, 72)  # Adjusted to match the saved model
        self.transformer = nn.Transformer(d_model=72, nhead=8, num_encoder_layers=6)  # Adjusted to match the saved model
        self.fc = nn.Linear(72, 72)  # Adjusted to match the saved model

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

    def get_embeddings(self, x):
        x = self.embed(x)
        return x


In [ ]:
# Load the state dictionary
model_path = 'saved/pretrain_base_hhar_20_120/hhar.pt'
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Print the keys of the state dictionary
print(state_dict.keys())


---------------------


In [15]:


# Load the configuration
with open('dataset/data_config.json', 'r') as f:
    config = json.load(f)

# Adjust based on the specific dataset you're working with
dataset_name = 'hhar_20_120'
#input_dim = config[dataset_name]['dimension']
#hidden_dim = 72  # This should match your model's configuration
#output_dim = len(config[dataset_name]['activity_label'])

# Load the pre-trained model
#model = LIMUBERTModel(input_dim, hidden_dim, output_dim)
model = LIMUBERTModel()

model_path = 'saved/pretrain_base_hhar_20_120/hhar.pt'
model.load_state_dict(state_dict)
model.eval()

# Function to run inference
def run_inference(model, data):
    model.eval()
    with torch.no_grad():
        data_tensor = torch.from_numpy(data).float()
        output = model(data_tensor)
        predictions = torch.argmax(output, dim=-1)
    return predictions.numpy()

def extract_segment(data, start_index, seq_len=120):
    """
    Extracts a 6-second segment from the data starting from start_index.
    
    Parameters:
        data (np.array): The dataset array of shape (N, W, F).
        start_index (int): The starting index for the segment.
        seq_len (int): The sequence length (default is 120).
    
    Returns:
        np.array: A segment of shape (1, seq_len, F).
    """
    end_index = start_index + 1
    segment = data[start_index:end_index, :seq_len, :]
    return segment

# Load your data
data_path = 'dataset/hhar/data_20_120.npy'
data = np.load(data_path)

# Extract a 6-second segment starting from index 0
start_index = 0
segment = extract_segment(data, start_index)

# Run inference on the extracted segment
predictions = run_inference(model, segment)

# Print the predictions
print(predictions)

RuntimeError: Error(s) in loading state_dict for LIMUBERTModel:
	Missing key(s) in state_dict: "embed.weight", "embed.bias", "transformer.encoder.layers.0.self_attn.in_proj_weight", "transformer.encoder.layers.0.self_attn.in_proj_bias", "transformer.encoder.layers.0.self_attn.out_proj.weight", "transformer.encoder.layers.0.self_attn.out_proj.bias", "transformer.encoder.layers.0.linear1.weight", "transformer.encoder.layers.0.linear1.bias", "transformer.encoder.layers.0.linear2.weight", "transformer.encoder.layers.0.linear2.bias", "transformer.encoder.layers.0.norm1.weight", "transformer.encoder.layers.0.norm1.bias", "transformer.encoder.layers.0.norm2.weight", "transformer.encoder.layers.0.norm2.bias", "transformer.encoder.layers.1.self_attn.in_proj_weight", "transformer.encoder.layers.1.self_attn.in_proj_bias", "transformer.encoder.layers.1.self_attn.out_proj.weight", "transformer.encoder.layers.1.self_attn.out_proj.bias", "transformer.encoder.layers.1.linear1.weight", "transformer.enc

In [11]:

# Instantiate the model
model = LIMUBERTModel()

# Path to your model file
model_path = 'saved/pretrain_base_hhar_20_120/hhar.pt'

# Load the state dictionary
state_dict = torch.load(model_path)

# Load the state dict into the model
model.load_state_dict(state_dict, strict=False)
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Set model to evaluation mode
model.eval()

LIMUBERTModel(
  (embed): Linear(in_features=6, out_features=72, bias=True)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=72, out_features=72, bias=True)
          )
          (linear1): Linear(in_features=72, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=72, bias=True)
          (norm1): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=72, out_features=72, bias=True)
          )
          (linear1):

In [13]:
# Function to run inference
def run_inference(model, data_segment):
    # Ensure data_segment is a torch tensor
    if not isinstance(data_segment, torch.Tensor):
        data_segment = torch.tensor(data_segment, dtype=torch.float32)
    
    # Add batch dimension if necessary
    if len(data_segment.shape) == 2:
        data_segment = data_segment.unsqueeze(0)
    
    # Prepare tgt tensor
    tgt = data_segment.clone()  # Modify this if needed based on model's requirements
    
    with torch.no_grad():
        output = model(data_segment, tgt)
    
    return output

def extract_segment(data, start_index, seq_len=120):
    """
    Extracts a 6-second segment from the data starting from start_index.
    
    Parameters:
        data (np.array): The dataset array of shape (N, W, F).
        start_index (int): The starting index for the segment.
        seq_len (int): The sequence length (default is 120).
    
    Returns:
        np.array: A segment of shape (1, seq_len, F).
    """
    end_index = start_index + 1
    segment = data[start_index:end_index, :seq_len, :]
    return segment

# Load your data
data_path = 'dataset/hhar/data_20_120.npy'
data = np.load(data_path)
print(data.shape)
# Extract a 6-second segment starting from index 0
start_index = 0
segment = extract_segment(data, start_index)



(9166, 120, 6)


In [14]:
run_inference(model, data)


TypeError: forward() takes 2 positional arguments but 3 were given